In [1]:
import re
import pymongo
import datetime
import pandas as pd
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)
base_url = "https://beta.bls.gov/dataQuery/"
starting_url = base_url + "find?q=data+scientists&r=100&st=0"
browser.visit(starting_url)
search_html = browser.html
soup_search = bs(search_html, 'html.parser')
data_query_results = soup_search.find_all("div", class_='dq-result-item')

url_dict = {"linkID":[],"description":[],"url":[]}
data_query_df = pd.DataFrame([])

last_link_on_page = data_query_results[-1].a.get('href')

text_parent = soup_search.find_all("div", class_='dq-page-links')
pages = text_parent[0].span.text.split()[-1]

# want to put number of pages into loop boundary range(pages)
for page in range(2):
    search_html = browser.html
    soup_search = bs(search_html, 'html.parser')
    data_query_results = soup_search.find_all("div", class_='dq-result-item')
    for result in data_query_results:
        result_url = result.a.get('href')
        link_title = result.label.get_text()
        link_id = result.input.get('value')
        # print(link_title)
        # print(link_id)
        if result_url != last_link_on_page:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
        else:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
            next_div = soup_search.find_all("div", class_='dq-next-page')
            next_url = base_url + next_div[0].a.get('href')
            print(next_url)
    browser.visit(next_url)

url_dict_df = pd.DataFrame(url_dict)

url_dict_df



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
There is no [mac64] chromedriver for browser 94.0.4606 in cache
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/justin/.wdm/drivers/chromedriver/mac64/94.0.4606.61]


https://beta.bls.gov/dataQuery/find?st=100&r=100&q=data+scientists&more=0


,linkID,description,url
0,OEUN000000051800015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
1,OEUN000000051820015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
2,OEUN000000051700015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
3,OEUN000000051800015209803,Hourly mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
4,OEUN000000051800015209804,Annual mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
...,...,...,...
195,OEUN000000061100315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
196,OEUN000000061130015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
197,OEUN000000061120215209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
198,OEUN000000061120315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...


In [3]:
browser.quit()

In [4]:
text_parent = soup_search.find_all("div", class_='dq-page-links')
last_page_number = int(text_parent[0].span.text.split()[-1])
print(last_page_number)

54


In [5]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

for url in range(len(url_dict["url"])):
    print(url_dict["url"][url])



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/justin/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051700015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209808
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209813
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209808
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209813
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000061100015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000056130015209801
https://beta.bls.gov/dataViewer/view/t

In [7]:
bls_state_data_df = pd.read_csv('state_M2020_dl.csv')
state_df = bls_state_data_df.loc[bls_state_data_df["AREA_TITLE"]=="Alabama", :]
job_in_state_df = state_df.loc[state_df["OCC_CODE"]=="15-2098", :]
job_in_state_df 

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
77,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,15-2098,Data Scientists and Mathematical Science Occup...,...,43.55,50.79,58.59,"56,510","74,460","90,590","105,650","121,870",NaN,NaN


In [8]:
bls_state_data_df

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,17.43,27.39,41.07,"18,690","24,060","36,250","56,980","85,430",NaN,NaN
1,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,45.73,64.58,91.89,"47,740","67,330","95,120","134,320","191,130",NaN,NaN
2,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,77.55,#,#,"49,480","97,930","161,290",#,#,NaN,NaN
3,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,48.64,73.58,#,"48,030","67,740","101,170","153,050",#,NaN,NaN
4,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1031,Legislators,...,*,*,*,"16,220","17,190","18,820","27,920","55,970",True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36080,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7021,Crane and Tower Operators,...,29.43,31.41,32.70,"54,630","57,100","61,220","65,340","68,020",NaN,NaN
36081,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7051,Industrial Truck and Tractor Operators,...,12.70,14.77,21.39,"23,570","24,560","26,410","30,720","44,480",NaN,NaN
36082,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7062,"Laborers and Freight, Stock, and Material Move...",...,13.96,17.49,19.94,"23,340","24,970","29,040","36,380","41,470",NaN,NaN
36083,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7064,"Packers and Packagers, Hand",...,11.76,12.31,15.31,"23,320","23,380","24,470","25,610","31,840",NaN,NaN
